In [35]:
import os
import glob
import filament
import datetime
import cmocean
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
from importlib import reload

In [45]:
reload(filament)

<module 'filament' from '/home/ctroupin/Projects/FNRS-UpwellingFilaments/Filament-Submesoscale/python/filament.py'>

In [37]:
datadir = "/data/SST/SouthAfrica/"
figdir = "/data/SST/SouthAfrica/figures"
visiblefile = "/data/Visible/VIIRS_20170902.tiff"
datafilelist = sorted(glob.glob(os.path.join(datadir, "*SST.nc")))

In [46]:
visibleim = filament.Visible()
visibleim.read_from(visiblefile)

In [40]:
if not(os.path.exists(figdir)):
    os.makedirs(figdir)

In [41]:
coordinates = (7., 19., -35., -23.)

In [42]:
m = Basemap(projection='merc', llcrnrlon=coordinates[0], llcrnrlat=coordinates[2],
            urcrnrlon=coordinates[1], urcrnrlat=coordinates[3],
            lat_ts=0.5 * (coordinates[2] + coordinates[3]), resolution='i')

In [80]:
for datafile in datafilelist[0:1]:
    sst = filament.SST()
    sst.read_from_oceancolorL2(datafile)
    sst.field = np.ma.masked_where(sst.qflag > 1, sst.field)
    titletext = sst.get_title(datafile)
    figname = os.path.join(figdir, sst.get_figname(datafile))
    make_plot_sst(sst, figname)

In [76]:
sst.field.min()

10.635

In [79]:
def make_plot_sst(sst, figname):
    fig = plt.figure(figsize=(8, 8))
    ax = plt.subplot(111)

    # Background: visible image grey
    m.pcolormesh(visibleim.lon, visibleim.lat, visibleim.array[:,:,0], latlon=True, 
             cmap=plt.cm.gray, zorder=1)
    pcm = m.pcolormesh(sst.lon, sst.lat, sst.field, latlon=True,
                 cmap=cmocean.cm.thermal, vmin=11., vmax=20., zorder=2)
    cb = plt.colorbar(pcm, extend="both")
    cb.set_label("$^{\circ}$C", rotation=0, ha="left")
    m.drawcoastlines(linewidth=0.25)
    #m.fillcontinents(zorder=4)
    m.drawmeridians(np.arange(coordinates[0], coordinates[1], 3.), labels=(0,0,1,0), 
                    linewidth=.5, fontsize=14, zorder=3)
    m.drawparallels(np.arange(coordinates[2], coordinates[3], 2.), labels=(1,0,0,0), 
                    linewidth=.5, fontsize=14, zorder=3)
    plt.savefig(figname, dpi=300, bbox_inches="tight")
    # plt.show()
    plt.close()